In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support import expected_conditions as EC

from concurrent.futures import ThreadPoolExecutor
import multiprocessing

from Functions.function import movies_for_category, categories, print_process, movies
from bs4 import BeautifulSoup
import requests



In [2]:
categorias = categories()

|---------------------Categorias capturadas correctamente.---------------------|


In [3]:

# Obtener la cantidad de núcleos disponibles en el sistema
num_nucleos = multiprocessing.cpu_count()

# Crear un diccionario para almacenar los resultados
resultados_dict = {}

with ThreadPoolExecutor(max_workers=num_nucleos) as executor:
    # Ejecutar la función para cada categoría en paralelo
    resultados = list(executor.map(movies_for_category, categorias))
    
    # Asignar los resultados al diccionario
    for categoria, resultado in zip(categorias, resultados):
        resultados_dict[categoria] = resultado

peliculas_total = []

for x,i in resultados_dict.items():
    for e in i:
        peliculas_total.append(e)
        
peliculas_total = set(peliculas_total)



|---Todas las peliculas de la categoria 'history' capturadas correctamente.---|
|--Todas las peliculas de la categoria 'biography' capturadas correctamente.--|
|---Todas las peliculas de la categoria 'fantasy' capturadas correctamente.---|
|----Todas las peliculas de la categoria 'family' capturadas correctamente.----|
|--Todas las peliculas de la categoria 'adventure' capturadas correctamente.--|
|--Todas las peliculas de la categoria 'animation' capturadas correctamente.--|
|----Todas las peliculas de la categoria 'crime' capturadas correctamente.----|
|----Todas las peliculas de la categoria 'action' capturadas correctamente.----|
|----Todas las peliculas de la categoria 'horror' capturadas correctamente.----|
|---Todas las peliculas de la categoria 'musical' capturadas correctamente.---|
|-----Todas las peliculas de la categoria 'news' capturadas correctamente.-----|
|--Todas las peliculas de la categoria 'reality_tv' capturadas correctamente.--|
|----Todas las peliculas de la cate

In [ ]:
peliculas_total_test = peluc

In [19]:
# Crear un diccionario para almacenar los resultados
json_pelioculas = {}

with ThreadPoolExecutor(max_workers=num_nucleos) as executor:
    # Ejecutar la función para cada categoría en paralelo
    resultados = list(executor.map(movies, peliculas_total))
    
    # Asignar los resultados al diccionario
    for pelicula, resultado in zip(peliculas_total, resultados):
        json_pelioculas[pelicula] = resultado


4585